In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
import pandas as pd
import polars as pl
from scripts.df_ct_combine import *

In [4]:
orders = df_ct_combine.combine()
orders['articles'] = orders['articles'].apply(list)
for i in range(len(orders)):
    orders.loc[i] = [orders['customer_id'][i], list(map(int, orders['articles'][i]))]

df_t = pd.read_csv('~/CIS_4496_Project/data/raw/transactions_train.csv')
df_c = pd.read_csv('~/CIS_4496_Project/data/raw/customers.csv')

In [4]:
clean_ar = pd.read_csv('~/CIS_4496_Project/data/clean_ar.csv')

def clean(arstr):
    return int(arstr.replace('frozenset({', '').replace('})', ''))

clean_ar['antecedents'] = list(map(clean, clean_ar['antecedents']))
clean_ar['consequents'] = list(map(clean, clean_ar['consequents']))

clean_ar = pl.from_pandas(clean_ar)

/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:3199: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


In [32]:
def predict(antecedent, rules, max_results= 12):
    
    preds = rules.filter(pl.col('antecedents') == antecedent)
    
    preds = preds['consequents']
    
    return preds[:max_results]

In [41]:
predict(df_t['article_id'][2], clean_ar).is_empty()

True

In [36]:
no_ct = list(set(df_c['customer_id']).symmetric_difference(set(np.unique(df_t['customer_id']))))
most_popular = pl.Series(list(df_t['article_id'].value_counts().index[0:12]))

In [55]:
submission = pl.DataFrame(schema={"customer_id": str , "prediction": pl.List(inner=pl.Int64)})

from datetime import datetime
print(datetime.now())

i = 0
df_num = 54
for c, alist in zip(orders['customer_id'][:50000], orders['articles'][:50000]):
    if (i % 1000 == 0):
        print(datetime.now(), i)
    i += 1
    
    predictor = False
    for a in alist:
        if predict(a, clean_ar).is_empty() == False:
            submission = submission.vstack(pl.DataFrame([[c], [predict(a, clean_ar)]], schema=submission.schema))
            predictor = True
            break
    if predictor == False:
        submission = submission.vstack(pl.DataFrame([[c], [most_popular]], schema=submission.schema))
        
    if i % 25000 == 0:
        submission.to_pandas().to_csv('~/CIS_4496_Project/data/submission/submission' + str(df_num) + '.csv', index=False)
        print('submission' + str(df_num) + ' saved.')
        df_num += 1
        submission = submission.clear()
    
for c in no_ct:
    if (i % 1000 == 0):
        print(datetime.now(), i)
    i += 1
    
    submission = submission.vstack(pl.DataFrame([[c], [most_popular]], schema=submission.schema))
    
    if i % 25000 == 0:
        submission.to_pandas().to_csv('~/CIS_4496_Project/data/submission/submission' + str(df_num) + '.csv', index=False)
        print('submission' + str(df_num) + ' saved.')
        df_num += 1
        submission = submission.clear()
    
submission.to_pandas().to_csv('~/CIS_4496_Project/data/submission/submission' + str(df_num) + '.csv', index=False)
print('submission' + str(df_num) + ' saved.')

2023-03-29 11:37:48.274880
2023-03-29 11:37:48.275854 0
2023-03-29 11:37:54.172313 1000
2023-03-29 11:38:01.365381 2000
2023-03-29 11:38:09.833239 3000
2023-03-29 11:38:19.651308 4000
2023-03-29 11:38:30.592100 5000
2023-03-29 11:38:42.919971 6000
2023-03-29 11:38:56.386056 7000
2023-03-29 11:39:11.415706 8000
2023-03-29 11:39:27.385078 9000
2023-03-29 11:39:44.694284 10000
2023-03-29 11:40:03.693512 11000
2023-03-29 11:40:23.974710 12000
2023-03-29 11:40:46.376362 13000
2023-03-29 11:41:10.329604 14000
2023-03-29 11:41:35.796007 15000
2023-03-29 11:42:03.511127 16000
2023-03-29 11:42:32.944017 17000
2023-03-29 11:43:04.206144 18000
2023-03-29 11:43:36.607202 19000
2023-03-29 11:44:10.279088 20000
2023-03-29 11:44:45.188653 21000
2023-03-29 11:45:21.411552 22000
2023-03-29 11:45:58.496651 23000
2023-03-29 11:46:36.771267 24000
submission54 saved.
2023-03-29 11:47:18.232342 25000
2023-03-29 11:47:24.209579 26000
2023-03-29 11:47:31.534850 27000
2023-03-29 11:47:39.938947 28000
2023-03-2

KeyboardInterrupt: 

In [76]:
from scripts import submission_combine
ar_submission = submission_combine.combine()

In [77]:
ar_submission

customer_id,prediction
str,list[str]
"""9733668ae674a1...","[""625818001"", ""610776002"", ... ""399223001""]"
"""973370352464f9...","[""634009003"", ""708021001"", ... ""706016002""]"
"""97383beabf6b02...","[""610776002"", ""620081001"", ... ""516712001""]"
"""973c939c49aa9a...","[""706016001"", ""706016002"", ... ""562245046""]"
"""974dd831c05a93...","[""669215004"", ""610776002"", ... ""554450001""]"
"""974ffc925761d2...","[""706016001"", ""706016002"", ""539723001""]"
"""97510a8afb16ae...","[""628930008"", ""706016001"", ... ""751471001""]"
"""975181834c5b45...","[""512251003"", ""610776002"", ... ""673677002""]"
"""97613d0fab4246...","[""706016001""]"


In [78]:
list(set(df_c['customer_id']).symmetric_difference(set(ar_submission['customer_id'])))

[]

In [79]:
ar_submission = ar_submission.to_pandas()
def convert(predStr):
    return predStr.replace('[', '').replace(']', '').replace(',', '').replace("'", '').replace('\n', '')

ar_submission['prediction'] = ar_submission['prediction'].apply(str).apply(convert)

In [80]:
ar_submission

,customer_id,prediction
0,9733668ae674a13fb4663f118d3139217fc5dc856009fd...,625818001 610776002 573716012 610776001 626816...
1,973370352464f952f0123bd0f2b2abbcf017cc2bdb564c...,634009003 708021001 634009008 484398001 564786...
2,97383beabf6b02fdea99a8f1c11e52d4e27aca40e98209...,610776002 620081001 620081006 610776001 372860...
3,973c939c49aa9a1e1cd765285c7495260521b46bfcc297...,706016001 706016002 399223029 706016003 562245...
4,974dd831c05a9364f94caa75e1bb2edb5ff5c74bec31ea...,669215004 610776002 610776001 669215006 706016...
...,...,...
1371975,971eeb671ada6db4774c4d26714227705506cae3ecc985...,610776001 678942001
1371976,9720d03350f27cb024fcfe5ce0877da46b786f57a70f41...,685687004 685687003 685687002 706016001 673677...
1371977,9728f0e7ab72a3c9e265d9a48a4871439a7993b9f2a62f...,189616006 189616008 189616001 706016001 706016...
1371978,9729ac874163fa6268d2527becd4e90704828a8034991c...,706016001 706016002 372860001 610776002 759871...


In [83]:
ar_submission.to_csv('~/CIS_4496_Project/data/submission/ar_submission.csv', index=False)